# Paralelização

Até agora o código que estamos construindo roda em série em nosso computador. Isso significa que cada instrução em nosso código é processada linearmente, uma após a outra.

Por exemplo, no caso de um `loop`, cada **etapa** do loop é executada em série. Vamos ver isso na prática.

## Códigos Seriais

Vamos começar analisando a **ordem de execução** de um loop simples:

In [ ]:
import time

In [ ]:
def dormir(x):
    '''
    Dormir x-segundos e retorna x.
    '''
    
    print(f'Dormindo por {x} segundos.')
    time.sleep(x)
    print(f'Retornando {x}')
    return x

In [ ]:
%%time
dormir(5)

Vamos construir uma lista e utilizar um `loop` para percorrer nossa lista:

In [ ]:
my_list = [1,2,3,4]

In [ ]:
%%time
for x in my_list:
    dormir(x)

O `loop` levou 10s para percorrer a lista pois percorreu ela em **série**: elemento a elemento, um após o outro.

Mesmo as técnicas de *programação funcional* executam o código da mesma maneira:

In [ ]:
%%time
list(map(dormir, [1, 2, 3, 4]))

## Códigos Paralelos

Uma das principais técnicas para aumentar a velocidade dos processadores no séc. XXI são os processadores com múltiplos *cores* (um processador contém múltiplas CPUs).

Cada *core* de um computador é linear: ele executa, em altissíma velocidade, instruções de forma linear - uma após a outra.

Os programas que construímos até hoje nas aulas são incapazes de utilizar múltiplos *cores* de processamento pois são lineares! Todos os `loops`, `applies`, `requests` ocorreram em série, de forma que, se nosso processador possui mais de um *core* não alavancamos toda sua capacidade computacional.

Vamos aprender agora como podemos converter nossos programas seriais em programas paralelos:

In [3]:
from multiprocessing import Pool, cpu_count
cpu_count()

8

A função `cpu_count()` nos mostra quantos *cores* nosso processador possui. Vamos utilizar a biblioteca `multiprocessing` para parelizar um loop.

## Criando um `Pool`

Um `Pool` é um objeto que coordena as tarefas que precisam ser executadas (*seu programa*) e como estas são alocadas nos diferentes *cores* do processador (*seu computador*).

Antes de mais nada, precisamos criar um `Pool`, determinando quantos *cores* esse *gerente* poderá utilizar.

In [4]:
pool = Pool(processes=cpu_count()-1)

In [5]:
pool

<multiprocessing.pool.Pool state=RUN pool_size=7>

### We'll `%%time` here to measure the velocity of this code in parallel.

However, if you run this code, watch what happens:

In [ ]:
#%%time

result = pool.map(dormir, my_list)
pool.terminate()

Infelizmente, o Jupyter Notebook tem algumas incosistências quando lidamos com códigos paralelizados. Para utilizar o nosso `Pool` em um Jupyter Notebook vamos apreender como criar uma biblioteca e importa-la!

In [6]:
import dormir

In [7]:
dormir.dormir_do_arquivo

<function dormir.dormir_do_arquivo(x)>

In [8]:
from dormir import dormir_do_arquivo

In [9]:
%%time
list(map(dormir_do_arquivo, [1, 2, 3, 4]))

Dormindo por 1 segundos.
Retornando 1
Dormindo por 2 segundos.
Retornando 2
Dormindo por 3 segundos.
Retornando 3
Dormindo por 4 segundos.
Retornando 4
CPU times: user 4.1 ms, sys: 1.75 ms, total: 5.85 ms
Wall time: 10 s


[1, 2, 3, 4]

In [16]:
pool.terminate()
pool = Pool(processes=cpu_count() - 1)
pool

<multiprocessing.pool.Pool state=RUN pool_size=7>

In [17]:
%%time
result = pool.map(dormir_do_arquivo, [1, 2, 3, 4])

Dormindo por 3 segundos.
Dormindo por 4 segundos.
Dormindo por 1 segundos.
Dormindo por 2 segundos.
Retornando 1
Retornando 2
Retornando 3
Retornando 4
CPU times: user 3.97 ms, sys: 3.05 ms, total: 7.02 ms
Wall time: 4.01 s


In [14]:
print(result)
pool.terminate()

[1, 2, 3, 4, 5, 6, 7]


O que aconteceu? O processamento **paralelo terminou em 4s**, comparado aos **10s do processamento em série**! O objeto `Pool` despachou cada uma das aplicações da função para um *core* diferente e as executou simultaneamente:

![image](images/parallel_vs_serial.webp)

### Nem todos os loops podem ser paralelos...

Para que um loop possa ser paralelizado, os resultados de cada *perna* do loop deve ser independente das outras *pernas*. Vamos construir um loop que não pode ser paralelizado:

In [18]:
minha_lista = [1, 2, 3, 4, 5]
fatorial = 1

for x in minha_lista:
    fatorial *= x

print(fatorial)

120


Cada *perna* do nosso `loop` **DEPENDE** da *perna* anterior! Logo, para executar a segunda etapa do `loop` precisamos executar a primeira, para executar a terceira, precisamos executar a segunda, e assim por diante.

Quais `loops` conseguimos paralelizar? Todos aqueles que podem ser escritos através de uma `list comprehensions` ou um `apply`.

## Utilizando `multiprocess`

Além de utilizar funções definidas externamente, podemos utilizar a biblioteca `multiprocess` no lugar da biblioteca `multiprocessing` para paralelizar nosso código.

In [1]:
!pip3 install multiprocess

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [19]:
from multiprocess import Pool, cpu_count

A sintáxe desta biblioteca é idêntica à biblioteca `multiprocessing`!

In [25]:
pool = Pool(processes=cpu_count() - 1)

In [21]:
%%time
result = list(map(lambda x:x**10000000, [1,2,3,4,5,6]))

CPU times: user 15.4 s, sys: 114 ms, total: 15.5 s
Wall time: 15.6 s


In [27]:
%%time
result = pool.map(lambda x:x**10000000, [1,2,3,4,5,6])

CPU times: user 16.4 ms, sys: 13.9 ms, total: 30.3 ms
Wall time: 6.18 s


In [23]:
pool.terminate()

## Códigos Assíncronos

Até agora todos os códigos que rodamos, mesmo os paralelos, foram processados de forma **síncrona**: enquanto o código é processado, o Python fica aguardando. Com processamento paralelo podemos construir códigos assíncronos

## What is asynchrony?

- `result.ready()`
- `result.wait()`
- `result.get()`

In [ ]:
pool = Pool(processes=cpu_count()-1)

In [ ]:
result = pool.map_async(my_sleep_from_file, [10, 10, 10, 10, 10, 10])

In [ ]:
print('Do something that doesn"t depend on result')
print('...')
print('Now the time came when the result is needed.')
result.wait()

result_list = result.get()
pool.terminate()
print(f'Now go on and use the results obtained - {result_list}')

In [ ]:
result.ready()

# Apêndice - Utilizando Paralelização em WebScrapping

In [ ]:
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
n_max = 51
my_range = range(1,n_max)

In [ ]:
%%time

for i in tqdm(my_range):
    response = requests.get(f'http://books.toscrape.com/catalogue/page-{i}.html')
    html=response.content
    soup = BeautifulSoup(html)
    titles=[s.find_all('a')[0]['title'] for s in soup.find_all('h3')]
    prices = [s.text for s in soup.find_all('p', attrs={'class':'price_color'})]
    stocks = [(True if s.text.strip()=='In stock' else False) for s in soup.find_all('p', attrs={'class':'instock availability'})]
    df_temp=pd.DataFrame({'Title':titles,'Price':prices,'Stock Availability':stocks})
    df_temp.to_csv(f'tmp/results_{i}.csv', index=False, sep=',')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def download(i):
    '''
    
    '''
    response = requests.get(f'http://books.toscrape.com/catalogue/page-{i}.html')
    html=response.content
    soup = BeautifulSoup(html)
    titles=[s.find_all('a')[0]['title'] for s in soup.find_all('h3')]
    prices = [s.text for s in soup.find_all('p', attrs={'class':'price_color'})]
    stocks = [(True if s.text.strip()=='In stock' else False) for s in soup.find_all('p', attrs={'class':'instock availability'})]
    df_temp=pd.DataFrame({'Title':titles,'Price':prices,'Stock Availability':stocks})
    df_temp.to_csv(f'tmp_par/results_{i}.csv', index=False, sep=',')

In [ ]:
pool = Pool(cpu_count())

In [ ]:
%%time

results = pool.map(download, tqdm(my_range))

In [ ]:
pool.terminate()

In [ ]:
def download_html(i):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    response = requests.get(f'http://books.toscrape.com/catalogue/page-{i}.html')
    html=response.content
    file = open(f'html_books_{i}.html','wb')
    file.write(response.content)

In [ ]:
import os

In [ ]:
os.mkdir('tmp_2')

In [ ]:
os.getcwd()

In [ ]:
os.chdir('tmp_2')

In [ ]:
os.getcwd()

In [ ]:
pool = Pool(cpu_count())
results = pool.map(download_html, tqdm(my_range))